In [ ]:
from notebook import notebookapp
from IPython.display import Markdown as md

servers = list(notebookapp.list_running_servers())
s = servers[0]['base_url'][1:] + 'voila/render/notebooks/path_finder_local.ipynb'

md("<b>Voila rendering of this notebook can be found through the link below.</b><br>[link](/{}).".format(s))

In [ ]:
import pandas as pd
from hdfs3 import HDFileSystem
import os
username = os.environ['JUPYTERHUB_USER']

# username = "jellouli" # UNCOMMENT if you haven't run schedule.ipynb and istdaten.ipynb

In [ ]:
# Read data from HDFS

In [ ]:
hdfs = HDFileSystem(user='ebouille') # impersonate ebouille to read the file
def read_hdfs(path):
    files = hdfs.glob(path)
    df = pd.DataFrame()
    for file in files:
        with hdfs.open(file) as f:
            df = df.append(pd.read_parquet(f))
    return df

In [ ]:
walking_edges = read_hdfs('/user/%s/final/parquet/walking_edges/*.parquet' % username)

In [ ]:
transport_edges = read_hdfs('/user/%s/final/parquet/transport_edges/*.parquet' % username)

In [ ]:
reachable_stops = read_hdfs('/user/%s/final/parquet/reachable_stops/*.parquet' % username)

In [ ]:
routes = read_hdfs('/user/%s/final/parquet/routes/*.parquet' % username)

In [ ]:
# Delay averages
default = read_hdfs('/user/%s/final/parquet/default/*.parquet' % username)
t_type_line = read_hdfs('/user/%s/final/parquet/t_type_line/*.parquet' % username)
t_type_line_hour = read_hdfs('/user/%s/final/parquet/t_type_line_hour/*.parquet' % username)
t_type_line_hour_stop = read_hdfs('/user/%s/final/parquet/t_type_line_hour_stop/*.parquet' % username)

In [ ]:
walking_edges['value'] = list(zip(walking_edges['end_vertex'], walking_edges['duration']))
walking_edges = walking_edges.groupby('start_vertex').agg({'value': set})\
                             .to_dict()['value']

In [ ]:
# Shortest Path

In [ ]:
# This function assumes a pandas dataframe with columns 'start_vertex', 'end_vertex', 'start_time', 'duration', 'route_id' 
# edges is a dataframe that represents the edges of the graph, 
# end_time should be in minutes
def latest_departure_paths(edges, end_stop, end_time, walking_edges):

    def edge_valid(u, v, time, dur, route_id): 
        next_edges = all_roads.get(v, [])
        
        # Did we reach the destination ?
        if v == end_stop and time + dur <= end_time: 
             return (time, v, route_id, dur)
        
        if u != end_stop and v != end_stop:
            for edge in next_edges[::-1]: # Traverse the list in reverse order because it's sorted by descending starting time
                time_v, next_v, route_id_v, dur_v = edge
                if (route_id == route_id_v and time + dur <= time_v) or (time + dur + transfer_time <= time_v): 
                    return (time, v, route_id, dur)            
            
            # Can we walk from v instead?
            for end_vertex, walking_duration in walking_edges.get(v, []):
                if end_vertex != u: # do not loop back to u
                    for next_next_edge in all_roads.get(end_vertex, []):
                        time_next = next_next_edge[0]
                        if time + dur <= time_next - walking_duration - transfer_time : # can we make in time to v in order to walk
                            return (time, v, route_id, dur)
                
        return None
                
    transfer_time = 2 # at least 2 minutes to transfer
    
    all_roads = {}
    start_time = end_time - 120 # look only at edges departing at most 2 hours before end_time
    
    all_roads[end_stop] = [(end_time, end_stop, '', 0)]
    
    edges = edges[(edges['start_time'] >= start_time) & (edges['start_time'] <= end_time)].sort_values(by=['start_time'], ascending=False).to_numpy()

    for row in edges:
        start_vertex, end_vertex, start, duration, route_id = row[:5]
        update = edge_valid(start_vertex, end_vertex, start, duration, route_id)
        if update:
            if start_vertex in all_roads:    
                all_roads[start_vertex].append(update)
            else:
                all_roads[start_vertex] = [update]
                        
    return all_roads

In [ ]:
from itertools import groupby
import numpy as np

# Journey reconstruction

# Returns a List of all possible edges that can be taken after edge 'edge'
def select_next(all_roads, walking_edges, edge, transfer_time=2):
    
    prev_time, node, prev_route_id, prev_duration = edge[:4]
    next_edges = all_roads.get(node, [])
    
    if prev_route_id == 'Walking': 
        # A walking edge encodes only one next trip at the node, stored as an index of the trip in all_roads[node] 
        return [next_edges[edge[-1]]]
    else:
        possible_edges = []
        # We may take a transport edge or a walking edge in this case
        next_edges = next_edges +  get_walking_edges(all_roads, walking_edges, node)
        min_departures = {}
        for edge_n in next_edges:
            t, v, route_id, dur = edge_n[:4]
            if route_id == 'Walking' and prev_time + prev_duration <= t:
                possible_edges.append(edge_n)
            elif prev_route_id == route_id and prev_time + prev_duration <= t:
                if (route_id, v) in min_departures and  min_departures[(route_id, v)][0] > t :
                    min_departures[(route_id, v)] = edge_n
                elif route_id not in min_departures:
                    min_departures[(route_id, v)] = edge_n
            elif prev_route_id != route_id and prev_time + prev_duration + transfer_time <= t:
                possible_edges.append(edge_n)
        possible_edges += min_departures.values()

        return possible_edges 

# Recursively iterate on all possible paths to the destination, and yield journeys as we find them 
def iterate_all_edges(all_roads, walking_edges, start_edges, end_stop, end_time, cur_journey):
    for edge in start_edges:
        t, v, route_id, dur = edge[:4]
        if v == end_stop and t + dur <= end_time: # have we reached the destination?
            yield cur_journey + [edge]
        elif v != end_stop:
            next_edges = select_next(all_roads, walking_edges, edge)
            
            for journey in iterate_all_edges(all_roads, walking_edges, next_edges, end_stop, end_time, cur_journey + [edge]):
                yield journey
                
# Compute the number of transfers as well as the total walking duration for a Journey
def get_journey_attributes(journey):
    walking_distance = sum([t[-2] for t in journey if t[2] == 'Walking'])
    transfers = len(set([t[2] for t in journey if t[2] != 'Walking']))
    return [transfers, walking_distance]

# Compute viable walks starting at node start_stop
# A walking edge u -> v holds information about the single trip to take from v
# This avoids unnecessary path computations
def get_walking_edges(all_roads, walking_edges, start_stop, transfer_time=2):
    possible_walking_edges = []
    for end_vertex, duration in walking_edges.get(start_stop, []):
        
        for index, edge in enumerate(all_roads.get(end_vertex, [])):
            time_next, node_next = edge[:2]
            if node_next != start_stop:
                possible_walking_edges.append((time_next - duration - transfer_time, end_vertex, 'Walking', duration, index))
    return possible_walking_edges
    
def generate_best_paths(all_roads, walking_edges, start_stop, end_stop, end_time):
    # add possible paths that start by walking
    start_edges = all_roads.get(start_stop, []) + get_walking_edges(all_roads, walking_edges, start_stop)
    if not start_edges: # not paths were found
        yield []
    else:
        # sort edges and group by start_time
        start_edges = sorted(start_edges, key=lambda x:x[0], reverse=True)
        start_edges = groupby(start_edges, key=lambda x:x[0])
        for start_time, edges in start_edges:
            # find all possible journeys that start at time start_time
            all_journeys = list(iterate_all_edges(all_roads, walking_edges, list(edges), end_stop, end_time, []))
            journey_attribs = np.array([get_journey_attributes(journey) for journey in all_journeys])
            sorted_indices = np.lexsort(journey_attribs.T)
            for i in sorted_indices:
                yield all_journeys[i]

In [ ]:
# Processes a list of edges (Journey) into a list of segments
def journey_to_segments (journey, start_stop):
    
    grouped_journey = [(route_id, list(edges)) for (route_id, edges) in  groupby(journey, lambda x: x[2])]
    segments = []
    for index, (route_id, edges) in enumerate(grouped_journey):
        
        start_time_s = edges[0][0]
        
        if route_id == 'Walking' and index > 0:
            # Start walking as soon as you get to the previous stop
            start_time_s = segments[-1][4] + segments[-1][5]
             
        end_stop_s = edges[-1][1]
        duration_s = edges[-1][0] + edges[-1][3] - edges[0][0]
        
        start_stop_s = start_stop if index == 0 else segments[index - 1][2]
            
        stops_s = [start_stop_s] + [t[1] for t in edges]
                
        segments.append((index + 1, start_stop_s, end_stop_s, route_id, start_time_s, duration_s, stops_s))
            
    return segments

In [ ]:
# Lookup dicts for human readable information

In [ ]:
stop_id_to_stop_dict = reachable_stops[['stop_id', 'stop_name']].set_index('stop_id').to_dict()['stop_name']
stop_id_to_lon_dict = reachable_stops[['stop_id', 'stop_lon']].set_index('stop_id').to_dict()['stop_lon']
stop_id_to_lat_dict = reachable_stops[['stop_id', 'stop_lat']].set_index('stop_id').to_dict()['stop_lat']

In [ ]:
routes['route_name'] = routes['route_desc'] + ' ' + routes['route_short_name']
route_id_to_route_name_dict = routes[['route_id', 'route_name']].set_index('route_id').to_dict()['route_name']
route_id_to_route_name_dict['Walking'] = 'Walk'

In [ ]:
# Delay computation

In [ ]:
# Dicts to speedup average delay lookup times
t_type_line_dict = t_type_line.set_index(['transport_type', 'line_name']).to_dict()['avg_delay']
t_type_line_hour_dict = t_type_line_hour.set_index(['transport_type', 'line_name', 'hour']).to_dict()['avg_delay']
t_type_line_hour_stop_dict = t_type_line_hour_stop.set_index(['transport_type', 'line_name', 'hour', 'stop_id']).to_dict()['avg_delay']
default_delay = default.values[0][0]

In [ ]:
# Extracts the type of transport from a route name, which may be 'Bus', 'Tram' or 'Train'. Otherwise returns 'Unknown'
def get_vehicle_type(route_name):
    first_word = route_name.split()[0]
    if first_word in ['Bus', 'Tram']:
        return first_word
    elif first_word in ['Eurocity', 'Eurostar', 'Extrazug', 'ICE', 'InterRegio', 'Intercity', 'Metro',\
                       'Nacht-Zug', 'RegioExpress', 'RegionalZug', 'S-Bahn', 'Schnellzug', 'TGV', 'Standseilbahn']:
        return 'Train'
    return 'Unknown'

# Makes a link between the istdaten route names and the schedule route name, by translating when available
def get_istdaten_line_name(route_name):
    route_name_parts = route_name.split(" ")
    if len(route_name_parts) >= 2:
        line_name = route_name_parts[0].replace("RegioExpress", "RE").replace("Intercity", "IC").replace("Extrazug", "EXT").replace("InterRegio", "IR").replace("S-Bahn", "S")
        line_number = route_name_parts[1].replace("-Y", "")
        if (line_name == "Bus" or line_name == "Tram" ):
            return line_number
        return line_name + line_number
    else:
        return route_name_parts[0]

def exponential_cdf(lambda_, x):
    return 1 - np.exp(- x / lambda_)

# Computes the probability of one journey
def compute_prob(segments, end_time):
    prob = 1.0
    num_segments = len(segments)
    
    for index, segment in enumerate(segments):
        
        max_delay = 0
        if segment[3] != 'Walking':
            if index + 1 < num_segments:
                if segments[index + 1][3] == 'Walking' :
                    if index + 2 < num_segments:
                        max_delay = segments[index + 2][4] - (segment[4] + segment[5] + segments[index + 1][5])
                else:
                    max_delay = segments[index + 1][4] - (segment[4] + segment[5])
            else:
                 max_delay =  end_time - (segment[4] + segment[5])                                          
            
            stop_id = segment[2]
            line_name =  get_istdaten_line_name(route_id_to_route_name_dict[segment[3]])
            hour = (segment[4] + segment[5]) // 60
            t_type = get_vehicle_type(route_id_to_route_name_dict[segment[3]])   
            
            if (t_type, line_name, hour, stop_id) in t_type_line_hour_stop_dict:
                lambda_ = t_type_line_hour_stop_dict[(t_type, line_name, hour, stop_id)]
            elif (t_type, line_name, hour) in t_type_line_hour_dict:
                lambda_ = t_type_line_hour_dict[(t_type, line_name, hour)]
            elif (t_type, line_name) in t_type_line_dict:
                lambda_ = t_type_line_hour_dict[(t_type, line_name)]
            else:
                lambda_ = default_delay
                
            if lambda_ != 0:
                prob *= exponential_cdf(lambda_, max_delay * 60)
                                                                                                                   
    return prob

In [ ]:
# Generate N paths 

In [ ]:
def generate_n_journeys(dep, dest, day, arrival_time, q, n=5):
    day_edges = transport_edges[transport_edges.weekday == day]
    all_roads = latest_departure_paths(day_edges, dest , arrival_time, walking_edges)
    generator = generate_best_paths(all_roads, walking_edges, dep, dest, arrival_time)
    
    journey_list = {}
    i = 1
    trials = 0
    while len(journey_list) < n:
        if trials == 200:
            return journey_list
        
        try:
            journey = next(generator)
        except StopIteration:
            return journey_list
                
        segments = journey_to_segments(journey, dep)
        journey_q = compute_prob(segments, arrival_time)
        
        if journey_q >= q:
            journey_list[i] = [seg + (journey_q,) for seg in segments]
            i += 1
        
        trials += 1
        
    return journey_list

In [ ]:
# Pretty print

In [ ]:
def time_to_minutes(timestamp):
        return int(timestamp[:2]) * 60 + int(timestamp[3:5])

In [ ]:
def minutes_to_timestamp(minutes):
    if minutes < 60:
        return str(minutes)
    return str(minutes // 60) + ':' + str(minutes % 60).zfill(2)

In [ ]:
def minutes_to_duration(minutes):
    if minutes < 60:
        return str(minutes) + ' min'
    return str(minutes // 60) + ' hr ' + str(minutes % 60) + ' min'

In [ ]:
def pretty_print_journey(journey):
    journey_str = '\n'.join(["%s : %s\n|\n| %s (%s min)\n| %s\n|\n%s : %s\n" % (minutes_to_timestamp(time), 
                                                                             stop_id_to_stop_dict[start],
                                                                             route_id_to_route_name_dict[route_id],
                                                                             minutes_to_timestamp(duration),
                                                                             ' -> '.join([stop_id_to_stop_dict[s] for s in stops]) if route_id != 'Walking' else '',
                                                                             minutes_to_timestamp(time + duration),
                                                                             stop_id_to_stop_dict[end]) for _, start, end, route_id, time, duration, stops in journey])
    return journey_str

In [ ]:
def journeys_to_str(journey_list):
    if len(journey_list) == 0:
        return "No possible path"

    res = []
    for i in journey_list.keys():
        segment_list = sorted(journey_list[i], key = lambda e: e[0])
        res.append(pretty_print_journey(segment_list))
    return '\n******************************\n'.join(res)

In [ ]:
# Visualization

In [ ]:
import plotly.graph_objects as go
from ipywidgets import interact

def map_plot():
    fig = go.Figure()

    fig.add_trace(go.Scattermapbox(
    ))

    fig.update_layout(
        hovermode='closest',
        mapbox=dict(
            style='carto-positron',
            bearing=0,
            pitch=0,
            zoom=11
        ),
        height=900,
        width=1200
    )

    return fig

In [ ]:
fig = map_plot()
stream_fig = go.FigureWidget(fig)
stream_data = stream_fig.data

def show_path(journey_segments):
    if len(journey_segments) == 0:
        return

    journey = sorted(journey_segments, key = lambda e: e[0])        
    
    departure_time = journey[0][4]
    arrival_time = journey[-1][4] + journey[-1][5]
    
    q = journey[0][-1]
    
    stops_per_segment = [seg[-2] for seg in journey]
    
    stops_per_segment_no_duplicates = [stops_per_segment[0]] + [ss[1:] for ss in stops_per_segment[1:]]

    stops = [item for sublist in stops_per_segment_no_duplicates for item in sublist]
    
    lats = [stop_id_to_lat_dict[s] for s in stops]
    lons = [stop_id_to_lon_dict[s] for s in stops]
    
    departure_stop = stops[0]
    arrival_stop = stops[-1]
    
    # Reset figure and center figure
    with stream_fig.batch_update():
        stream_fig.data = stream_fig.data[:1]
        stream_fig.update_layout(mapbox_center=dict(lat = (max(lats)+min(lats))/2, lon = (max(lons)+min(lons))/2))
        stream_fig.update_layout(mapbox_zoom=11)
        stream_fig.update_layout(legend_traceorder="reversed", legend_valign='top')
        stream_fig.update_layout(title_text='%s (%s) -> %s (%s) - %s' % (stop_id_to_stop_dict[departure_stop],
                                                                         minutes_to_timestamp(departure_time),
                                                                         stop_id_to_stop_dict[arrival_stop],
                                                                         minutes_to_timestamp(arrival_time),
                                                                         'CL = {:.2%}'.format(q)))
    
    
    stops_per_route = [(seg[3], seg[4], seg[5], seg[-2]) for seg in journey]
    
    for route_id, time, duration, stops in reversed(stops_per_route):
        stop_names = [stop_id_to_stop_dict[s] for s in stops]
        
        trace_name = '<b>' + route_id_to_route_name_dict[route_id] + '</b><br>%s %s<br>%s' % (stop_names[0],
                                                                                              minutes_to_timestamp(time),
                                                                                              minutes_to_duration(duration))
        
        if route_id != 'Walking':
            trace_name += ' (%d stops)' % (len(stops)-1)
        trace_name += '<br>%s %s' % (stop_names[-1], minutes_to_timestamp(time + duration))
        
        with stream_fig.batch_update():
            stream_fig.add_trace(go.Scattermapbox(
                lat=[stop_id_to_lat_dict[s] for s in stops],
                lon=[stop_id_to_lon_dict[s] for s in stops],
                mode='lines+markers',
                marker=go.scattermapbox.Marker(
                    size=6,
                ),
                opacity=1,
                hovertext=stop_names,
                name=trace_name,
                hoverinfo="text+name",
                hoverlabel_namelength=-1,
                showlegend=True
            ))
            
    return stream_fig

In [ ]:
stop_name_to_id = sorted(list({v:k for k,v in stop_id_to_stop_dict.items()}.items()))
weekdays = [(s, s.lower()) for s in ['Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday']]
times = [(minutes_to_timestamp(m), m) for m in range(9*60, 20*60+1, 5)]
percentages = list(reversed([("{:.0%}".format(p/20), p/20) for p in range(21)]))

In [ ]:
from ipywidgets import interact_manual, Dropdown, HBox, Label, interact
from IPython.display import display

style = {'description_width': '100px'}

dep_widget = Dropdown(options=stop_name_to_id, value=None, description= 'Departure stop:', style=style)
dest_widget = Dropdown(options=stop_name_to_id, value=None, description='Destination stop:', style=style)
day_widget = Dropdown(options=weekdays, value=None, description='Day:', style=style)
arr_time_widget = Dropdown(options=times, value=None, description='Arrival time:', style=style)
Q_widget = Dropdown(options=percentages, value=None, description='Confidence level:', style=style)


def ask_for_input(dep, dest, day, arr_time, Q):
    if (dep and dest and day and arr_time and Q) is None:
        print("Please set all the parameters.")
        return
    
    
    journeys = generate_n_journeys(dep, dest, day, arr_time, Q)
    
    if len(journeys) == 0:
        print('Sorry, no journeys were found for the desired parameters.')
        return
    
    if len(journeys) < 5:
        print('Only %s journeys were found by the algorithm after trying 200 different routes' % len(journeys))
    
    @interact(i=Dropdown(options=journeys.keys(), description='Journey number:', style=style))
    def show_fig(i):
        return show_path(journeys[i])
    

im = interact_manual(ask_for_input, dep=dep_widget, dest=dest_widget, day=day_widget, arr_time=arr_time_widget, Q=Q_widget)
im.widget.children[5].description = 'Search'


@interact
def f():
    im